<a href="https://colab.research.google.com/github/SvitlanaSm/A-B-testing/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Етап 1. Розрахунок статистичної значущості.

Аналізувати статистичну значущість за допомогою онлайн-калькуляторів не дуже зручно. Тому доопрацьовуєм розрахунок статистичної значущості через Python скрипт.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
# Connecting Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Changing work folder
%cd /content/drive/MyDrive/mate/colab/data_loading

# Uploading dataset
df = pd.read_csv("bq-results-20250510-112706-1746876439677.csv")
df.head()

Mounted at /content/drive
/content/drive/MyDrive/mate/colab/data_loading


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


In [ ]:
# Info about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  object
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  int64 
 6   test_group  800996 non-null  int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 55.0+ MB


In [ ]:
# Count each event
event_counts = df['event_name'].value_counts()
print(event_counts)

event_name
session                107210
session_start          106242
page_view              101907
user_engagement         94520
first_visit             81621
scroll                  73643
view_promotion          61695
view_item               44869
session with orders     25892
new account             22389
view_search_results     14310
add_shipping_info       11961
begin_checkout          11959
select_item             11816
add_to_cart             11762
select_promotion         9044
add_payment_info         8048
click                    1997
view_item_list            111
Name: count, dtype: int64


In [ ]:
# Creating a pivot table to count values by event_name and test_group
pivot_table = pd.pivot_table(df, values= "value", index = "event_name", columns = "test_group", aggfunc = "count")
print(pivot_table)

test_group               1      2
event_name                       
add_payment_info      4018   4030
add_shipping_info     5949   6012
add_to_cart           5916   5846
begin_checkout        5948   6011
click                  959   1038
first_visit          40845  40776
new account          11255  11134
page_view            50936  50971
scroll               36918  36725
select_item           5906   5910
select_promotion      4529   4515
session              53587  53623
session with orders  12936  12956
session_start        53121  53121
user_engagement      47303  47217
view_item            22556  22313
view_item_list          60     51
view_promotion       30879  30816
view_search_results   7191   7119


In [ ]:
# Unique test numbers
test_numbers = df["test"].unique()
sorted_test_numbers = np.sort(test_numbers)
print(sorted_test_numbers)

[1 2 3 4]


In [ ]:
# Unique test groups
test_groups = df["test_group"].unique()
sorted_test_groups = np.sort(test_groups)
print(sorted_test_groups)

[1 2]


In [ ]:
# Delete " " in event_name
df["event_name"] = df["event_name"].str.replace(" ", "_")
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new_account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new_account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new_account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new_account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new_account,1


In [ ]:
# Unique event_names
events_unique = df["event_name"].unique()
print(events_unique)

['new_account' 'session_with_orders' 'session' 'page_view'
 'user_engagement' 'session_start' 'add_payment_info' 'first_visit'
 'view_promotion' 'scroll' 'begin_checkout' 'view_item'
 'view_search_results' 'select_item' 'add_shipping_info' 'add_to_cart'
 'click' 'select_promotion' 'view_item_list']


Розраховуємо значущість в розрізі тестів. Розрахунок зробили через масиви значень та цикли, без статичного закріплення кількості метрик, та їх значень.

In [ ]:
# Сalculating statistical significance for 4 metrics
metrics = {'add_payment_info/session': ('add_payment_info', 'session'),
'add_shipping_info/session': ('add_shipping_info', 'session'),
'begin_checkout/session': ('begin_checkout', 'session'),
'new_account/session': ('new_account', 'session')}
result = []

for test_num in sorted_test_numbers:
   test_data = df[df["test"] == test_num]

   for metric_name, (numerator_event, denominator_event) in metrics.items():

      numerator = test_data[(test_data['event_name'] == numerator_event)]
      denominator = test_data[(test_data['event_name'] == denominator_event)]

      numerator_count = test_data[(test_data['event_name'] == numerator_event) & (test_data['test_group'] == 2)]['value'].sum()
      denominator_count = test_data[(test_data['event_name'] == denominator_event) & (test_data['test_group'] == 2)]['value'].sum()

      numerator_count_control = test_data[(test_data['event_name'] == numerator_event) & (test_data['test_group'] == 1)]['value'].sum()
      denominator_count_control = test_data[(test_data['event_name'] == denominator_event) & (test_data['test_group'] == 1)]['value'].sum()

      conversion_rate = (numerator_count / denominator_count)
      conversion_rate_control = (numerator_count_control / denominator_count_control)

      metric_change = (conversion_rate - conversion_rate_control) / conversion_rate_control * 100
      z_stat, p_value = sm.stats.proportions_ztest( [numerator_count, numerator_count_control], [denominator_count, denominator_count_control] )
      if p_value < 0.05:
          significant = "TRUE"
      else:
          significant = "FALSE"
      result.append({
          "test_number": test_num,
          "metric": metric_name,
          "numerator_event": numerator_event,
          "denominator_event": denominator_event,
          "numerator_count": numerator_count,
          "denominator_count": denominator_count,
          "conversation_rate": round(conversion_rate, 3),
          "numerator_count_control": numerator_count_control,
          "denominator_count_control": denominator_count_control,
          "conversation_rate_control": round(conversion_rate_control, 3),
          "metric_change": round(metric_change, 3),
          "z_stat": round(z_stat, 3),
          "p_value": round(p_value, 3),
          "significant": significant
      })

result_df = pd.DataFrame(result)
result_df.to_csv("ab_test_results_new.csv", index=False)
result_df.head()

,test_number,metric,numerator_event,denominator_event,numerator_count,denominator_count,conversation_rate,numerator_count_control,denominator_count_control,conversation_rate_control,metric_change,z_stat,p_value,significant
0,1,add_payment_info/session,add_payment_info,session,2229,45193,0.049,1988,45362,0.044,12.542,3.925,0.000,TRUE
1,1,add_shipping_info/session,add_shipping_info,session,3221,45193,0.071,3034,45362,0.067,6.560,2.604,0.009,TRUE
2,1,begin_checkout/session,begin_checkout,session,4021,45193,0.089,3784,45362,0.083,6.661,2.979,0.003,TRUE
3,1,new_account/session,new_account,session,3681,45193,0.081,3823,45362,0.084,-3.354,-1.543,0.123,FALSE
4,2,add_payment_info/session,add_payment_info,session,2409,50244,0.048,2344,50637,0.046,3.577,1.241,0.215,FALSE


У результаті отримали всі необхідні дані для подальшої візуалізації:  
* conversation_rate,  
* metric_change,  
* z_stat,  
* p_value,  
* significant

In [ ]:
# Save csv file
from google.colab import files
files.download("ab_test_results_new.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Етап 2. Візуалізація в Tableau.

Зробили візуалізацію файлу з етапу 1. Дашборд показуває значення основних 4 метрик та де вони змінються статистично, а де ні. Є присутній фільтр за номером тесту.  
[Дашборд](https://public.tableau.com/views/ABtest_17435129853620/ABtest?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)